In [158]:
import numpy as np
import matplotlib.pyplot as plt
import datetime
import re
import json
from elasticsearch import Elasticsearch, exceptions as es_exceptions
from elasticsearch.helpers import scan
es = Elasticsearch(hosts=[{'host':'atlas-kibana.mwt2.org', 'port':9200}],timeout=60)

In [159]:
#define function to print a few job information
def print_info(job):
    
    print('\n')    
    errors = ['pilot','brokerage','taskbuffer','sup','jobdispatcher','ddm','exe']
    #print(job)
    
    print('pandaid ' + str(job['_source']['pandaid']))
    print('modificationtime ' + str(job['_source']['modificationtime']))
    print('site '    + str(job['_source']['computingsite']))
    
    for e in errors:
        errorcode = e + 'errorcode'
        exitcode = job['_source'][errorcode] 
      
        if exitcode:
            print(errorcode + ' ' + str(exitcode))  
            errordiag = e +'errordiag'   
            print(j['_source'][errordiag])   


In [160]:
#define function to debug job information
def debug(job):
        
    errors = ['pilot','brokerage','taskbuffer','sup','jobdispatcher','ddm','exe']
    #print(job)
    pandaid=int(job['_source']['pandaid'])
    
    if (pandaid==3155163997): #3186999624):  #3155206788):
        print('pandaid ' + str(job['_source']['pandaid']))
        print('modificationtime ' + str(job['_source']['modificationtime']))
        print('site '    + str(job['_source']['computingsite']))
    
        for e in errors:
            errorcode = e + 'errorcode'
            exitcode = job['_source'][errorcode] 
      
            if exitcode:
                print(errorcode + ' ' + str(exitcode))  
                errordiag = e +'errordiag'   
                print(j['_source'][errordiag])   


In [161]:
#define function to filter on time
def time_filter(indices, last_days=1, pattern=''):
    if last_days == 0:
        return ["jobs_archive_*"]
    filtered = []
    if pattern:
        for i in indices:
            if pattern in i:
                filtered.append(i.rstrip())
        return filtered
    today = datetime.date.today()
    filtered = []
    datefmt = '%Y-%m-%d'
    for i in indices:
        day = re.sub(r'jobs_archive_', '', i).rstrip()
        #print(day)
        day = datetime.datetime.strptime(day, datefmt).date()
        diff = today - day
        if diff.days < last_days:
            filtered.append(i.rstrip())
    return filtered

In [162]:
#get job archive indices from ES
indices = es.cat.indices(index="jobs_archive_*", h="index", request_timeout=600).split('\n')
indices = sorted(indices)
indices = [x for x in indices if x != '']
if 'jobs_archive_2016_status' in indices:
    indices.remove('jobs_archive_2016_status')
#remove data due to central problem
if 'jobs_archive_2016-12-29' in indices:
    indices.remove('jobs_archive_2016-12-29')
#print(indices)

In [247]:
# use e.g. last_days=7 or pattern='2016-02' (no wildcard !)
NDAYS=''  #NDAYS=150
PATTERN = '2016-03' #PATTERN=''
ind = time_filter(indices, last_days=NDAYS, pattern=PATTERN)
ind = ','.join(ind)
print(ind)


jobs_archive_2016-03-01,jobs_archive_2016-03-02,jobs_archive_2016-03-03,jobs_archive_2016-03-04,jobs_archive_2016-03-05,jobs_archive_2016-03-06,jobs_archive_2016-03-07,jobs_archive_2016-03-08,jobs_archive_2016-03-09,jobs_archive_2016-03-10,jobs_archive_2016-03-11,jobs_archive_2016-03-12,jobs_archive_2016-03-13,jobs_archive_2016-03-14,jobs_archive_2016-03-15,jobs_archive_2016-03-16,jobs_archive_2016-03-17,jobs_archive_2016-03-18,jobs_archive_2016-03-19,jobs_archive_2016-03-20,jobs_archive_2016-03-21,jobs_archive_2016-03-22,jobs_archive_2016-03-23,jobs_archive_2016-03-24,jobs_archive_2016-03-25,jobs_archive_2016-03-26,jobs_archive_2016-03-27,jobs_archive_2016-03-28,jobs_archive_2016-03-29,jobs_archive_2016-03-30,jobs_archive_2016-03-31


In [248]:
#Try either simple query
query = "jobstatus:failed AND processingtype:gangarobot-pft"

#or add it to body
body = {
    "size": 2,
    "_source": ["computingsite", "pandaid", "modificationtime", "*errorcode", "*errordiag"], #returns only certain fields
    "query": { 
        "bool" : {
            "filter" : [
                { "match" : { "processingtype":"gangarobot-pft" } },
                { "match" : { "jobstatus":"failed" } },
              ],
           #"must" :                 
           #     {"match": {"processingtype":"gangarobot-pft"}},
           # "should" :
           #     {"match": {"jobstatus":"failed"}}
     
        }
    }}

In [249]:
#test query with search -> gets only a limited number of records (size=x). Cannot be used for big searches x>10000
res = es.search(index=ind, q=query, size = 2, request_timeout=600, 
                _source=["computingsite", "pandaid", "modificationtime", "*errorcode", "*errordiag"])
#res = es.search(index=ind, body=body, request_timeout=600)

jobs = res['hits']['hits']

#print(json.dumps(jobs))
#print a few information
for j in jobs:
    print_info(j)
    
#save to file
with open('data_few.txt', 'w') as outfile:  
    json.dump(jobs, outfile)
outfile.close()    



pandaid 2797638432
modificationtime 2016-03-17T01:14:21
site UNIBE-LHEP


pandaid 2797630345
modificationtime 2016-03-17T01:16:08
site UNIBE-LHEP


In [250]:
#count hits
count =(es.count(index=ind, q=query, request_timeout=600))
print('hits: '+str(count['count']))

#use scroll to get all hits (allows unlimted result queries)
scroll = scan(es, index=ind, q=query, scroll='5m', timeout="5m", size=100, _source=["computingsite", "pandaid", "modificationtime", "*errorcode", "*errordiag"])

#create filename
if (NDAYS==''):
    FILENAME = 'data_'+PATTERN+'.txt'
else:
    FILENAME = 'data_last'+LASTDAYS+'.txt'

print(FILENAME)
outfile = open(FILENAME, 'w') 

#write results to file
i = 0   
jobs = []
for result in scroll:
    #print(result['_source']['pandaid'])
    #print_info(result)
    jobs.append(result)
    #if i<1: print_info(result)
    i = i+1
    if not i%10000:  print('processing hit '+str(i)+'...')

json.dump(jobs, outfile)         
outfile.close()
print('saved '+str(i)+' results') 

#check written results
infile = open(FILENAME, 'r')
savedjobs = json.load(infile)
print('check: reading in '+str(len(savedjobs))+' hits')
infile.close()

print_info(savedjobs[0])
print_info(jobs[0])

hits: 43260
data_2016-03.txt
processing hit 10000...
processing hit 20000...
processing hit 30000...
processing hit 40000...
saved 43260 results
check: reading in 43260 hits


pandaid 2804382616
modificationtime 2016-03-22T01:05:26
site INFN-GENOVA
piloterrorcode 1099
None


pandaid 2804382616
modificationtime 2016-03-22T01:05:26
site INFN-GENOVA
piloterrorcode 1099
None


In [179]:
i = 0
for j in jobs:
    debug(jobs[i])
    i = i+1
    if not i%10000:  print('processing hit '+str(i)+'...')

processing hit 10000...
processing hit 20000...
processing hit 30000...
processing hit 40000...
processing hit 50000...
processing hit 60000...
processing hit 70000...
processing hit 80000...
processing hit 90000...
processing hit 100000...
processing hit 110000...
processing hit 120000...
processing hit 130000...
processing hit 140000...
processing hit 150000...
processing hit 160000...
processing hit 170000...
processing hit 180000...
processing hit 190000...
processing hit 200000...
processing hit 210000...
processing hit 220000...
processing hit 230000...
processing hit 240000...
processing hit 250000...
processing hit 260000...
processing hit 270000...
processing hit 280000...
processing hit 290000...
processing hit 300000...
processing hit 310000...
processing hit 320000...
processing hit 330000...
processing hit 340000...
processing hit 350000...
processing hit 360000...
processing hit 370000...
processing hit 380000...
processing hit 390000...
processing hit 400000...
processin

In [31]:
#Example of more refined queries
#better to use filters if the score is not needed (faster)
s = {
    "size": 1,
    "_source": ["computingsite", "pandaid", "modificationtime", "*errorcode", "*errordiag"], #returns only certain fields
    "query": { 
        "bool" : {
            "filter" : [
                { "match" : { "processingtype":"gangarobot-pft" } },
                { "match" : { "jobstatus":"failed" } },
              ],
           #"must" :                 
           #     {"match": {"processingtype":"gangarobot-pft"}},
           # "should" :
           #     {"match": {"jobstatus":"failed"}}
     
        }
    },
    "aggs": {  #build aggregation
        "models": 
            {"terms": { "field": "jobstatus" } }
    }    
}

res = es.search(index=ind, body=s, request_timeout=12000)

print(res) 

jobs = res['hits']['hits']
print('check: reading in '+str(len(jobs))+' hits')
#print_info(jobs[0])
print(jobs[0])

{'aggregations': {'models': {'sum_other_doc_count': 0, 'doc_count_error_upper_bound': 0, 'buckets': [{'key': 'failed', 'doc_count': 141998}]}}, 'took': 14, 'hits': {'total': 141998, 'max_score': 0.0, 'hits': [{'_id': '3124983243', '_type': 'jobs_data', '_source': {'taskbuffererrorcode': 0, 'jobdispatchererrorcode': 0, 'ddmerrordiag': None, 'computingsite': 'ATLAS_OPP_OSG-MIT_CMS', 'brokerageerrorcode': 0, 'exeerrorcode': 0, 'modificationtime': '2016-12-14T02:46:42', 'brokerageerrordiag': None, 'superrorcode': 0, 'pandaid': 3124983243, 'superrordiag': None, 'jobdispatchererrordiag': None, 'piloterrordiag': 'Job killed by signal 15: Signal handler has set job result to FAILED, ec = 1201', 'ddmerrorcode': 0, 'piloterrorcode': 1201, 'exeerrordiag': None, 'taskbuffererrordiag': None}, '_score': 0.0, '_index': 'jobs_archive_2016-12-14'}]}, '_shards': {'total': 155, 'failed': 0, 'successful': 155}, 'timed_out': False}
check: reading in 1 hits
{'_id': '3124983243', '_type': 'jobs_data', '_sour